In [8]:
# Import pandas
import pandas as pd
import os

# Build the relative path from notebook to Excel file
data_path = os.path.join("..", "data", "processed", "faa_enforcement_merged.xlsx")
output_path = "../data/processed/faa_enforcement_cleaned_test.xlsx"
# Load the cleaned dataset
df = pd.read_excel(data_path)

# Show basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3308 entries, 0 to 3307
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CASE NUMBER      3307 non-null   object 
 1   NAME             3305 non-null   object 
 2   ENTITY TYPE      3305 non-null   object 
 3   DATE KNOWN       3304 non-null   object 
 4   ACTION           3304 non-null   object 
 5   SANCTION AMOUNT  2684 non-null   object 
 6   SANCTION         3305 non-null   object 
 7   CASE TYPE        3303 non-null   object 
 8   CLOSED DATE      3303 non-null   object 
 9   MASTER #         2 non-null      object 
 10  c95              73 non-null     float64
 11  c710             73 non-null     float64
dtypes: float64(2), object(10)
memory usage: 310.3+ KB


In [9]:
#Quick scan through the file see a few problems:
#       - Extra 3 column that need to be drop 
#       - One row is almost empty missing case number missing name ...
#       - Date Known and CLOSSED DATE have difference format depend on the quarter 

# Drop unwanted/extra columns if they exist
extra_columns = ["MASTER #", "c95", "c710"]
df = df.drop(columns=[col for col in extra_columns if col in df.columns])

# Drop rows missing CASE NUMBER
df = df.dropna(subset=["CASE NUMBER"])

# Strip whitespace from string columns only
str_cols = df.select_dtypes(include="object").columns
for col in str_cols:
    df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)

# Fix casing inconsistencies
for col in ["ACTION", "SANCTION", "CASE TYPE"]:
    if col in df.columns:
        df[col] = df[col].str.upper()

# Normalize DATE KNOWN and CLOSED DATE to datetime format (YYYY-MM-DD)
df["DATE KNOWN"] = pd.to_datetime(df["DATE KNOWN"], errors="coerce").dt.date
df["CLOSED DATE"] = pd.to_datetime(df["CLOSED DATE"], errors="coerce").dt.date

# Drop fully empty rows
df = df.dropna(how="all")
# Reset index
df = df.reset_index(drop=True)


C:\Users\dcuon\AppData\Local\Temp\ipykernel_23204\3568791237.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE KNOWN"] = pd.to_datetime(df["DATE KNOWN"], errors="coerce").dt.date
C:\Users\dcuon\AppData\Local\Temp\ipykernel_23204\3568791237.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["CLOSED DATE"] = pd.to_datetime(df["CLOSED DATE"], errors="coerce").dt.date


In [10]:
df.info()
for col in df.columns:
    print(f"\n📊 Value counts for: {col}")
    print(df[col].value_counts(dropna=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3307 entries, 0 to 3306
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CASE NUMBER      3307 non-null   object
 1   NAME             3305 non-null   object
 2   ENTITY TYPE      3305 non-null   object
 3   DATE KNOWN       3303 non-null   object
 4   ACTION           3304 non-null   object
 5   SANCTION AMOUNT  2683 non-null   object
 6   SANCTION         3304 non-null   object
 7   CASE TYPE        3303 non-null   object
 8   CLOSED DATE      3303 non-null   object
dtypes: object(9)
memory usage: 232.7+ KB

📊 Value counts for: CASE NUMBER
CASE NUMBER
2014WP170043    4
2010EA610006    3
2019NM410003    3
2006SO150227    3
2012NM090010    3
               ..
2023SO670002    1
2024WP910004    1
2024SW910017    1
2024EA910011    1
2024WP910030    1
Name: count, Length: 2980, dtype: int64

📊 Value counts for: NAME
NAME
FEDERAL EXPRESS CORP            94
AMERICAN

In [11]:
# Columns to check (exclude 'SANCTION AMOUNT')
columns_to_check = [col for col in df.columns if col != "SANCTION AMOUNT"]

# Filter rows where any of these columns have nulls
rows_with_nulls = df[df[columns_to_check].isnull().any(axis=1)]

# Display the result
print(rows_with_nulls)
#Decide to remove 
df_cleaned = df.dropna(subset=columns_to_check)

       CASE NUMBER                 NAME       ENTITY TYPE  DATE KNOWN  \
87    2009NM070066  SCENIC AVIATION INC  A/C or COMM OPER  2009-03-23   
1181  2011NM710052                  NaN               NaN         NaT   
1182  2012EA700034      ROYAL AIR MAROC    FOREIGN AIR CA         NaT   
2242  2013AL030112      LEPPING, KURT M  A/C or COMM OPER         NaT   
2548  2019GL130007                  NaN               NaN         NaT   

                        ACTION SANCTION AMOUNT         SANCTION CASE TYPE  \
87               CONSENT ORDER           10000  SANCTION WAIVED       NaN   
1181                       NaN             NaN              NaN       NaN   
1182                       NaN             NaN              NaN       NaN   
2242  ORD ASSESS CIVIL PENALTY            5000          DOLLARS     OTHER   
2548                       NaN             NaN              NaN       NaN   

     CLOSED DATE  
87           NaT  
1181         NaT  
1182         NaT  
2242  2015-08-12  
254

In [12]:
# Save cleaned file
df_cleaned.to_excel(output_path, index=False)
print(f"✅ Cleaned and formatted dates. Output saved to: {output_path} with {len(df)} rows.")

✅ Cleaned and formatted dates. Output saved to: ../data/processed/faa_enforcement_cleaned_test.xlsx with 3307 rows.


In [13]:
df_cleaned.info()
for col in df_cleaned.columns:
    print(f"\n📊 Value counts for: {col}")
    print(df[col].value_counts(dropna=False))

<class 'pandas.core.frame.DataFrame'>
Index: 3302 entries, 0 to 3306
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CASE NUMBER      3302 non-null   object
 1   NAME             3302 non-null   object
 2   ENTITY TYPE      3302 non-null   object
 3   DATE KNOWN       3302 non-null   object
 4   ACTION           3302 non-null   object
 5   SANCTION AMOUNT  2681 non-null   object
 6   SANCTION         3302 non-null   object
 7   CASE TYPE        3302 non-null   object
 8   CLOSED DATE      3302 non-null   object
dtypes: object(9)
memory usage: 258.0+ KB

📊 Value counts for: CASE NUMBER
CASE NUMBER
2014WP170043    4
2010EA610006    3
2019NM410003    3
2006SO150227    3
2012NM090010    3
               ..
2023SO670002    1
2024WP910004    1
2024SW910017    1
2024EA910011    1
2024WP910030    1
Name: count, Length: 2980, dtype: int64

📊 Value counts for: NAME
NAME
FEDERAL EXPRESS CORP            94
AMERICAN AIRL

In [14]:
entity_type_mapping = {
    "A/C OR COMM OPER": "A/C or COMM OPER",
    "CERTIFICATE SHCOOL": "CERTIFICATE SCHOOL",
    "FOREIGN AIR CA": "FOREIGN AIR CAR",
    "APPROVD RPR STA": "APPROVD REPAIR STA",
    "APPROVED RPR STN": "APPROVD REPAIR STA",
    "AIR CARRIER ON DMAND": "AIR CARRIER ON DEMAND",
    "ARPT OPN/INSP": "AIRPORT OPERATOR",
    "COMP PROD": "COMPONENT PRODUCTION",
    "COMM OPER & PART 125": "A/C or COMM OPER",
    # You can add more if needed
}
df["ENTITY TYPE"] = df["ENTITY TYPE"].replace(entity_type_mapping)

In [ ]:
# Mapping for ACTION normalization
action_mapping = {
    "ORDER ASSESSING CIVIL PENALTY": "ORD ASSESS CIVIL PENALTY",
    "ORDER ASSESS CIVIL PENALTY": "ORD ASSESS CIVIL PENALTY",
    "ORD ASSESS CVIIL PENALTY": "ORD ASSESS CIVIL PENALTY",
    "ORD ASSESS CIVIL PENALTY CONSOLIDATE D CASE": "ORD ASSESS CIVIL PENALTY",

    "ORD ASSESS CP HMT": "ORD ASSESS CP HMT",
    "ORD ASSESS CP HMT AC": "ORD ASSESS CP HMT",

    "ORD ASSESS CP FA ACT": "ORD ASSESS CP FA ACT",
    "CP FA ACT": "ORD ASSESS CP FA ACT",


    "CERTIFICATE REVOCATION .": "CERTIFICATE REVOCATION",
    "CERTIFICATE REVOCATION 1": "CERTIFICATE REVOCATION",
    "CERT REVOKE": "CERTIFICATE REVOCATION",

    "CERTIFICATE SUSPENSION": "CERTIFICATE SUSPENSION",
    "CERT SUSPEND": "CERTIFICATE SUSPENSION",
}

# Normalize ACTION column
df["ACTION"] = df["ACTION"].replace(action_mapping)

In [ ]:
# Mapping for SANCTION normalization
sanction_mapping = {
    # Consolidated Case variants
    "CONSOLIDATE D CASE": "CONSOLIDATED CASE",
    "CONSOLIDAT ED CASE": "CONSOLIDATED CASE",
    "CONSOLID ATED CASE": "CONSOLIDATED CASE",
    "CONSOLIDA TED CASE": "CONSOLIDATED CASE",

    # Indefinite Duration variants
    "INDEFINIT E DURATION": "INDEFINITE DURATION",
    "INDEFINTE DURATION": "INDEFINITE DURATION",
}

# Normalize SANCTION column
df["SANCTION"] = df["SANCTION"].replace(sanction_mapping).astype(str).str.strip()

In [ ]:
# Mapping for CASE TYPE normalization
case_type_mapping = {
    "DRUG TESTING": "DRUG TESTING",
    "MAINTENANCE": "MAINTENANCE",
    "FLT OPNS": "FLT OPNS",
    "HAZ MAT": "HAZ MAT",
    "RECORDS/RPTS": "RECORDS/RPTS",
    "RECORDS/RPT S": "RECORDS/RPTS",
    "OTHER": "OTHER",
    "QUAL CONTROL": "QUAL CONTROL",
    "TRNG-FLT CREW": "TRNG-FLT CREW",
    "TRNG-OTHER": "TRNG-OTHER",
    "AIRCRAFT ALTR": "AIRCRAFT ALTR",
    "ACFT ALTR": "AIRCRAFT ALTR",  
    "ARPT OPN/INSP": "ARPT OPN/INSP",
    "ARPT SURF/SAFE": "ARPT SURF/SAFE",
    "CRASH/FIRE/RES": "CRASH/FIRE/RES",
    "TYPE DESGN DATA": "TYPE DESGN DATA",
    "REVOCATIO N OTHER": "REVOCATION OTHER",
    "REVOCATI ON OTHER": "REVOCATION OTHER",
    "REVOCATIO N FLT OPNS": "REVOCATION FLT OPNS",
    "REVOCATI ON FLT OPNS": "REVOCATION FLT OPNS",
    "REVOCATIO N MAINTENANCE": "REVOCATION MAINTENANCE",
    "REVOCATI ON MAINTENANCE": "REVOCATION MAINTENANCE",
    "REVOCATIO N TYPE DESGN DATA": "REVOCATION TYPE DESGN DATA",
    "REVOCATIO N RECORDS/RPTS": "REVOCATION RECORDS/RPTS",
    "HAZ PER/PROP": "HAZ PER/PROP",
    "HAZ AIR NAV": "HAZ AIR NAV",
    "SECURITY": "SECURITY",
    "MEDICAL": "MEDICAL"
}

# Apply normalization
df["CASE TYPE"] = df["CASE TYPE"].replace(case_type_mapping).astype(str).str.strip()


In [18]:
df_cleaned.info()
for col in df_cleaned.columns:
    print(f"\n📊 Value counts for: {col}")
    print(df[col].value_counts(dropna=False))

<class 'pandas.core.frame.DataFrame'>
Index: 3302 entries, 0 to 3306
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CASE NUMBER      3302 non-null   object
 1   NAME             3302 non-null   object
 2   ENTITY TYPE      3302 non-null   object
 3   DATE KNOWN       3302 non-null   object
 4   ACTION           3302 non-null   object
 5   SANCTION AMOUNT  2681 non-null   object
 6   SANCTION         3302 non-null   object
 7   CASE TYPE        3302 non-null   object
 8   CLOSED DATE      3302 non-null   object
dtypes: object(9)
memory usage: 258.0+ KB

📊 Value counts for: CASE NUMBER
CASE NUMBER
2014WP170043    4
2010EA610006    3
2019NM410003    3
2006SO150227    3
2012NM090010    3
               ..
2023SO670002    1
2024WP910004    1
2024SW910017    1
2024EA910011    1
2024WP910030    1
Name: count, Length: 2980, dtype: int64

📊 Value counts for: NAME
NAME
FEDERAL EXPRESS CORP            94
AMERICAN AIRL